In [ ]:
import pandas as pd
from PIL import Image
import numpy as np

In [ ]:
attns = pd.read_feather('/fast/rsna-breast/tables/attn_scores_all_atomic_sweep_134.feather')

In [ ]:
def getTile(ptID, imgID, row, col):
    fn = f'/fast/rsna-breast/tiles/224/{int(ptID)}/{int(imgID)}_{int(row)}_{int(col)}.png'
    pil = Image.open(fn)
    return pil


In [ ]:
getTile(0,0,0,0)

In [ ]:
top100=attns.sort_values('attention', ascending=False).head(100)
top = top100.iloc[0]
top100

In [ ]:
top100r=attns.sort_values('raw', ascending=False).head(100)
topr = top100r.iloc[0]
top100r

In [ ]:
top = top100.iloc[0]
top

In [ ]:
getTile(top.ptID, top.imgID, top.row, top.col)

In [ ]:
topr.astype(int)

In [ ]:
getTile(topr.ptID, topr.imgID, topr.row, topr.col)

In [ ]:
def mosaic(ptID, imgID, row, col):
    subset = attns[attns.imgID==imgID]
    print(subset)

mosaic(topr.ptID, topr.imgID, topr.row, topr.col)

In [ ]:
subset = attns[attns.imgID==1480395667]
subset

In [ ]:
subset.row.max(), subset.col.max()

In [ ]:
# width, height
def assembleFull(ptID, imgID):
    #ptID, imgID = 6654, 1480395667
    subset = attns[attns.imgID==imgID]
    dst = Image.new('RGB', (224*subset.col.max(), 224*subset.row.max()))
    for rn, R in subset.iterrows():
        tile = getTile(ptID, imgID, R.row, R.col)
        dst.paste(tile, (int(224*R.col), int(224*R.row)))
    return dst

In [ ]:
attns.row.max(), attns.col.max()

In [ ]:
labels[['patient_id','image_id']].value_counts()

In [ ]:
from glob import glob
from PIL import Image
import os
# width, height
def assembleFull(ptID, imgID):
    #ptID, imgID = 6654, 1480395667
    pat = f'/fast/rsna-breast/newtiles/224/{ptID}/{imgID}_*_*.png'
    tiles = glob(pat)
    maxRow=0
    maxCol=0
    L = []
    for tileF in tiles:
        #print(tileF)
        _, fn = os.path.split(tileF)
        fn, _ = os.path.splitext(fn)
        _, row, col = fn.split('_')
        row, col = int(row), int(col)
        maxRow = max(maxRow, row)
        maxCol = max(maxCol, col)
        tile = Image.open(tileF)
        L.append((row, col, tile))
    dst = Image.new('RGB', (224*(maxCol+1), 224*(maxRow+1)))
    for row, col, tile in L:
        dst.paste(tile, (int(224*col), int(224*row)))
    return dst

In [ ]:
cr = cancer.iloc[28]
print(cr)
assembleFull(cr.patient_id, cr.image_id)

In [ ]:
Rassemble(attns[attns.imgID==cr.image_id].sort_values('attention', ascending=False).iloc[1])

In [ ]:
cr = cancer.iloc[209]
print(cr)
assembleFull(cr.patient_id, cr.image_id)

In [ ]:
Rassemble(attns[attns.imgID==cr.image_id].sort_values('attention', ascending=False).iloc[0])

In [ ]:
cr = cancer.iloc[2]
print(cr)
assembleFull(cr.patient_id, cr.image_id)

In [ ]:
cr = cancer.iloc[2]
print(cr)
assembleFull(cr.patient_id, cr.image_id)

In [ ]:
labels[labels.patient_id==21923]

In [ ]:
cr = cancer.iloc[245]
print(cr)
assembleFull(cr.patient_id, cr.image_id)

In [ ]:
Rassemble(attns[attns.imgID==cr.image_id].sort_values('attention', ascending=False).iloc[10])

In [ ]:
assembleFull(6654, 1480395667)

In [ ]:
assembleFull(6654, 1480395667)

In [ ]:
labels = pd.read_csv('/fast/rsna-breast/train.csv')
cancer = labels[labels.cancer==1]


In [ ]:
cr = cancer.iloc[0]
print(cr)
assembleFull(cr.patient_id, cr.image_id)

In [ ]:
attns[attns.imgID==cr.image_id].sort_values('attention', ascending=False)

In [ ]:
Rassemble(attns[attns.imgID==cr.image_id].sort_values('attention', ascending=False).iloc[8])

In [ ]:
top100

In [ ]:
assembleFull(22376,387009159)

In [ ]:
assembleFull(22376,387009159)

In [ ]:
assembleFull(37035,1563009599)

In [ ]:
assembleFull(37035,1563009599)

In [ ]:
top100

In [ ]:
assembleFull(65389,240295884)

In [ ]:
assembleFull(65389,240295884)

In [ ]:
# dataframe implementation
def assembleROI(ptID, imgID, row, col):
    #ptID, imgID = 6654, 1480395667
    q = (attns.row)>row-2
    subset = attns[attns.imgID==imgID]
    dst = Image.new('RGB', (224*subset.col.max(), 224*subset.row.max()))
    for rn, R in subset.iterrows():
        tile = getTile(ptID, imgID, R.row, R.col)
        dst.paste(tile, (int(224*R.col), int(224*R.row)))
    return dst

In [ ]:
R.astype(int)

In [ ]:
# lazy filesystem implementation
def assembleROI(ptID, imgID, row, col):
    dst = Image.new('RGB', (224*3, 224*3))
    def paste(r,c):
        try:
            tile = getTile(ptID,imgID,row+r,col+c)
            dst.paste(tile, (224*(1+c), 224*(1+r)))
        except:
            print('missing')
    paste(-1,-1)
    paste(-1,0)
    paste(-1,1)

    paste(0,-1)
    paste(0,0)
    paste(0,1)

    paste(1,-1)
    paste(1,0)
    paste(1,1)

    

    return dst

R = top100r.iloc[0]
assembleROI(R.ptID, R.imgID, R.row, R.col)

In [ ]:
def Rassemble(R):
    return assembleROI(R.ptID, R.imgID, R.row, R.col)
Rassemble(top100.iloc[0])

In [ ]:
Rassemble(top100.iloc[1]).size

In [ ]:
224*3/2

In [ ]:
from torch.utils.data import Dataset

class RegionDataset(Dataset):
    def __init__(self, df, argparse):
        self.args = argparse
        self.df = df.reset_index()
        self.df = self.df.astype(int)

    def __len__(self):
        return len(self.df)

    def getTile(self, ptID, imgID, row, col):
        fn = f'/fast/rsna-breast/tiles/224/{int(ptID)}/{int(imgID)}_{int(row)}_{int(col)}.png'
        pil = Image.open(fn)
        return pil

    def assembleRegion(self, R):
        # lazy filesystem implementation
        R = R.astype(int)
        dst = Image.new('L', (224 * 3, 224 * 3))

        def paste(r, c):
            try:
                tile = self.getTile(R.ptID, R.imgID, R.row + r, R.col + c)
                dst.paste(tile, (224 * (1 + c), 224 * (1 + r)))
            except:
                print('missing')

        for r in [-1,0,1]:
            for c in [-1,0,1]:
                paste(r,c)

        return dst

    def __getitem__(self, item):
        R = self.df.iloc[item]
        regionIMG = self.assembleRegion(R)
        target = R.target
        return regionIMG, target, R.raw

top100r['target'] = 0
dataset = RegionDataset(top100r, None)
img, _, _ = dataset[19]
img

In [ ]:
labels = pd.read_csv('/fast/rsna-breast/train.csv')[['image_id','cancer']]
topEachImage.merge(labels, left_on='imgID', right_on='image_id').cancer.value_counts()


In [ ]:
#top5eachImage = attns.sort_values('attention', ascending=False).groupby('imgID').head(5)
topEachImage = attns.sort_values(['imgID','attention'], ascending=False).groupby('imgID').head(10)

In [ ]:
topEachImage.head(20)

In [ ]:
top5eachImage.groupby('imgID').sum()

In [ ]:
attns[attns.imgID==2147415608].sort_values('attention', ascending=False)

In [ ]:
top5eachImage.sort_values(['ptID','imgID', 'row', 'col', 'attention']).head(30)

In [ ]:
from torchvision import transforms as T
from torch.utils.data import Dataset
class RegionDataset(Dataset):
    def __init__(self, df, argparse=None, validation=False):
        self.args = argparse
        self.df = df.reset_index()
        self.df = self.df.astype(int)

        normalize = dict(mean=[0.5], std=[0.25])
        if validation:
            transform = [
                #T.RandomResizedCrop(size=224 * 2),
                T.CenterCrop(size=2*224)
            ]
        else:
            transform = [
                # T.RandomAffine(90.0),
                T.ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25),
                T.RandomAffine(90, shear=(-20.0, 20.0), scale=(0.95, 1.05)),
                T.CenterCrop(size=2*224),
                #T.RandomResizedCrop(size=224*2),
                T.RandomHorizontalFlip(p=0.5),
                T.RandomVerticalFlip(p=0.5),
                #T.RandomApply([color_jitter], p=cj_prob),
                #T.RandomGrayscale(p=random_gray_scale),
                #GaussianBlur(
                #    kernel_size=kernel_size * input_size_,
                #    prob=gaussian_blur),
            ]

        #normalize = dict(mean=[0.5], std=[0.25])
        transform += [
            T.ToTensor(),
            T.Normalize(mean=normalize['mean'], std=normalize['std'])      # FIXME
        ]

        transform = T.Compose(transform)
        self.transform = transform


    def __len__(self):
        return len(self.df)

    def getTile(self, ptID, imgID, row, col):
        fn = f'/fast/rsna-breast/tiles/224/{int(ptID)}/{int(imgID)}_{int(row)}_{int(col)}.png'
        pil = Image.open(fn)
        return pil

    def assembleRegion(self, R):
        # lazy filesystem implementation
        R = R.astype(int)
        dst = Image.new('L', (224 * 3, 224 * 3))

        missing = 0
        def paste(r, c):
            tile = self.getTile(R.ptID, R.imgID, R.row + r, R.col + c)
            dst.paste(tile, (224 * (1 + c), 224 * (1 + r)))

        for r in [-1, 0, 1]:
            for c in [-1, 0, 1]:
                try: paste(r, c)
                except FileNotFoundError:
                    missing += 1

        print(f'missing {missing} tiles')

        return dst

    def __getitem__(self, item):
        R = self.df.iloc[item]
        regionIMG = self.assembleRegion(R)
        regionIMG = self.transform(regionIMG)
        print(type(regionIMG))
        #a = np.asarray(regionIMG)
        a = regionIMG.numpy()
        print(a.min(),a.mean(), a.max())
        target = R.target.astype(np.float32)
        return regionIMG, target, R.raw


topEachImage['target']=1
dataset = RegionDataset(topEachImage)
dataset[33][0]